# Traffic assignment: scenario 1

First of all, we need to import gubori. We can do so in one of two ways:

* ``import gurobipy`` 
* ``from gurobipy import *``

In [1]:
from gurobipy import *

### Parameters
-----------------

We then need to setup our parameters. Here, we define two parameters:

* nPaths: the number of paths available, which is equal to 3 (north, south, center).
* nVehicles: the number of vehicles that need to travel, which in our case is 100.

In [2]:
nPaths=3 
nVehicles=100

### Model
-----
Next, we create the model. We can optionally provide it with a name. 

Note: the model we will create is *non-convex*: hence, we define that here. This is rare, and we will only need it in this model.

In [3]:
model=Model("traffic_assignment")
model.Params.NonConvex=2

Using license file C:\Users\yggdrasilyuan\gurobi.lic
Academic license - for non-commercial use only - expires 2023-10-05
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


### Variables
-----

And then we add **integer** variables for each of the flows and **continuous** variables for each of the times in the **3 paths**. Note how we define their types as `GRB.INTEGER` and `GRB.CONTINUOUS`, respectively. If we omit the type, then Gurobi will assume a continuous variable for us.

In [4]:
x=model.addVars(nPaths, vtype=GRB.INTEGER) 
t=model.addVars(nPaths, vtype=GRB.CONTINUOUS)

### Objective
-----
Now, we add an expression (called `obj` for objective function) and set it as the objective function of our model. Recall that the average time can be calculated as: $$avg=\frac{t_0\cdot x_0+t_1\cdot x_1+t_2\cdot x_2}{nVehicles}.$$

In [5]:
obj=(t[0]*x[0]+t[1]*x[1]+t[2]*x[2])/nVehicles
model.setObjective(obj)

### Constraints
-----

Continuing, we add the constraints. We have two types of constraints here:

1. the number of vehicles on the paths needs to sum up to nVehicles=100.
2. the time of each path depends on the number of vehicles using it.

Or, in mathematical terms:

1. $x_0+x_1+x_2=100$;
2. $t_0=\frac{x_0+x_2}{100}+3;~~~t_1=\frac{x_1+x_2}{100}+3;~~~t_2=\frac{x_0+x_1+2\cdot x_2}{100}+2.25$

In [6]:
model.addConstr(x[0]+x[1]+x[2]==nVehicles)

<gurobi.Constr *Awaiting Model Update*>

In [7]:
model.addConstr(t[0]==(x[0]+x[2])/100+3)
model.addConstr(t[1]==(x[1]+x[2])/100+3)
model.addConstr(t[2]==(x[0]+x[1]+2*x[2])/100+2.25)

<gurobi.Constr *Awaiting Model Update*>

### Optimizing
----
The `optimize()` method will begin the optimization process.

In [8]:
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 6 columns and 13 nonzeros
Model fingerprint: 0xdb325763
Model has 3 quadratic objective terms
Variable types: 3 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-02, 2e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+02]
Presolve time: 0.00s
Presolved: 11 rows, 10 columns, 28 nonzeros
Presolved model has 3 bilinear constraint(s)
Variable types: 7 continuous, 3 integer (0 binary)

Root relaxation: objective 3.000000e+00, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.00000    0    2          -    3.00000      -     -    0s
H    0     0                       3.50000

### Retrieving the solution
----
To access the results (*if* there is an optimal or feasible solution), we use `var.X`. For example here `x[0].X` will reveal the flow on the first path. If I do `t[0].X` then I will get the time on the first path.

In [9]:
print("Path\t\tTime\tVehicles")
print("-"*35)
for i in range(nPaths):
    print("Path "+str(i+1)+": \t"+str(round(t[i].X,2))+"\t"+str(round(x[i].X,2)))

Path		Time	Vehicles
-----------------------------------
Path 1: 	3.5	50.0
Path 2: 	3.5	50.0
Path 3: 	3.25	-0.0


We observe that the average time is 3.5 minutes, and the optimal solution lets half the population go south and half the population go north. Note also how the fastest path has no cars...

## Recall that we could write the formulation without the $t$ variables...

In [10]:
model=Model("traffic_assignment_without_t")
x=model.addVars(nPaths, vtype=GRB.INTEGER)
obj=((x[0]+x[2])/100+3)*x[0]+((x[1]+x[2])/100+3)*x[1]+((x[0]+x[1]+2*x[2])/100+2.25)*x[2]
model.setObjective(obj)
model.addConstr(x[0]+x[1]+x[2]==nVehicles)
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xa961d390
Model has 5 quadratic objective terms
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+00]
  QObjective range [2e-02, 4e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Presolved model has 5 quadratic objective terms
Variable types: 0 continuous, 3 integer (0 binary)
Found heuristic solution: objective 423.7600000

Root relaxation: objective 3.500000e+02, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     350.0000000  350.00000  0.00%     -   

In [11]:
t={}
t[0]=(x[0].X+x[2].X)/100+3
t[1]=(x[1].X+x[2].X)/100+3
t[2]=(x[0].X+x[1].X+2*x[2].X)/100+2.25

## Note how $t$ is *not* a variable now!

print("Path\t\tTime\tVehicles")
print("-"*35)
for i in range(nPaths):
    print("Path "+str(i+1)+": \t"+str(round(t[i],2))+"\t"+str(round(x[i].X,2)))

Path		Time	Vehicles
-----------------------------------
Path 1: 	3.5	50.0
Path 2: 	3.5	50.0
Path 3: 	3.25	-0.0
